In [1]:
!pip install easyocr opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 24.9 MB/s eta 0:00:00


In [2]:
import easyocr
import cv2
import re
import numpy as np

In [17]:
from google.colab import files
uploaded = files.upload()


Saving AHD-0425-PA-0007561_JITENDRA_TRIVEDI_DS_28-04-2025_1019-21_AM.pdf_page_9[1].png to AHD-0425-PA-0007561_JITENDRA_TRIVEDI_DS_28-04-2025_1019-21_AM.pdf_page_9[1].png


In [20]:
reader = easyocr.Reader(['en'])

image_name = list(uploaded.keys())[0]
image = cv2.imread(image_name)

results = reader.readtext(image)

ocr_text = " ".join([res[1] for res in results])

print("Extracted OCR Text:")
print(ocr_text)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Extracted OCR Text:
PLATELET INDICES _ 125000 117000 209000 PLATELET COUNT {cumm {cumm {cumm PLATELE T INDICES 10.9 @L 13.7@ 122@L PDW PLATELET INDICES MPV 10.6 fL 10.8 fL 10.41L PLATELE T' INDICES . P 30.0 % 32.3 % 28.3 % LCR PLATELET INDICES PCT 0.13 % 0.13 % 0.22 % PSSR MO Mild Mild Mild Anisocytosi Anisocytosi Anisocytosi Mild Mild s, Mild Microcytosi Microcytosi Microcytosi Mild s, Mild S, Mild Hypochrom Hypochrom Hypochromi ia, few ia, 4 Few pencil cells pencil cells seen seen PSSW MO Wichin Mild Mild Normal leucocytosis Polymorpho Limits nuclear Leucocytosi $ with mild left shin, PERIPHERAL SMEAR Mild Mild Adequate STUDY PLATELETS Reduction, Reduction, PERIPHERAL SMEAR Not Not Not STUDY PARASITE detected detected detected PERIPHERAL SMEAR Suggested STUDY Note Fe supplement Close Follow up for Hb HEMOGLOBIN 1L.0 9.6 9.6 RBC COUNT 4,83 421 3.98 mil/cumm mil/cumm mil/cumm PACKED CELL 35. % 30.2 % 29.5 % VOLUME(PCV) RBC INDICES MCV 73.08 @, 71,.73@L. 74.12@, RBC INDICES MCH 22.77 pg

In [11]:
# Dictionary to store processed lines for each image
processed_lines_per_image = {}

for filename, ocr_text in all_ocr_results.items():
    # Convert to lowercase
    text_lower = ocr_text.lower()

    # Split into lines (basic segmentation)
    # Using '.' as a delimiter to split sentences, but be aware this is basic
    lines = [line.strip() for line in text_lower.split('.') if line.strip()]
    processed_lines_per_image[filename] = lines

print("--- Processed Lines Per Image ---")
for filename, lines in processed_lines_per_image.items():
    print(f"File: {filename}")
    for line in lines:
        print(f"- {line}")
    print("\n")

--- Processed Lines Per Image ---


In [13]:
keywords = ["reduce", "avoid", "increase", "restrict", "advised"]

doctor_advice_per_image = {}

for filename, lines_list in processed_lines_per_image.items():
    advice_for_image = []
    for line in lines_list:
        for word in keywords:
            if word in line:
                advice_for_image.append(line.strip())
                break
    doctor_advice_per_image[filename] = advice_for_image

print("Filtered Doctor Advice Per Image:")
for filename, advice in doctor_advice_per_image.items():
    print(f"File: {filename}")
    if advice:
        for item in advice:
            print(f"- {item}")
    else:
        print("- No specific advice found")
    print("\n")

Filtered Doctor Advice Per Image:


In [14]:
rule_mapping = {
    "sugar": "Low sugar diet",
    "oil": "Low fat diet",
    "oily": "Low fat diet",
    "fried": "Low fat diet",
    "salt": "Low sodium diet",
    "vegetable": "High fiber diet",
    "vegetables": "High fiber diet"
}


In [21]:
diet_rules = []

# Iterate through the doctor_advice_per_image dictionary
for filename, advice_list in doctor_advice_per_image.items():
    for advice_item in advice_list:
        for key in rule_mapping:
            if key in advice_item:
                diet_rules.append(rule_mapping[key])

# Remove duplicates
diet_rules = list(set(diet_rules))

print("Final Structured Diet Rules:")
diet_rules

Final Structured Diet Rules:


[]